In [1]:
#library import
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
import os, random, json
spark=SparkSession.builder.master("local").appName("studentmarkpractice").getOrCreate()

In [2]:
class unification_to_refinement:
    
    """refinement"""
    
    def __init__(self,file_parameter):
        
        """function to get input parameters to read the file from local path"""
        self.file_parameter=file_parameter
        
    def read_parquet(self):
        return spark.read.format(self.file_parameter['src_format']).option("header","True").load(self.file_parameter['src_path'])
    
    def write_parquet(self,df):
        df.write.format(self.file_parameter['dest_format']).mode("overwrite").option("path",self.file_parameter['dest_path']).saveAsTable("school_unified.{0}".format(self.file_parameter['view_name']))
    
    def create_view(self,df):
        df.createOrReplaceTempView(self.file_parameter['view_name'])
    
    def open_dir(self):
        os.chdir(self.file_parameter['dest_path'])
        dest_file_path=f"{self.file_parameter['view_name']}.parquet"
        for file in os.listdir():
        # Check whether file is in text format or not
            if file.endswith(".parquet"):
                src_file_path = f"{file}"
                os.rename(src_file_path,dest_file_path)
            else:
                src_file_path = f"{file}"
                os.remove(src_file_path)
                
    def main_block(self):
        
        """main execution block"""
        df=self.read_parquet()
        self.create_view(df)
    

In [4]:
if __name__=='__main__':
    file_parameter1={
        "src_format":"parquet",
        "src_path":"C:/Git_Repo/big_data_project/data_files/unified/school_unified/overallreport/school.parquet",
        "view_name":"school"
    }
    
    file_parameter2={
        "dest_format":"parquet",
        "dest_path":"C:/Git_Repo/big_data_project/data_files/refinement/school_refinement/highestmark",
        "view_name":"highestmark"
    }
    
    spark.sql("create database if not exists {0}".format("school_unified"))
    unified_obj1=unification_to_refinement(file_parameter1)
    unified_obj2=unification_to_refinement(file_parameter2)
    unified_obj1.main_block()
    highmark_df=spark.sql("select * from (select *,dense_rank() over(partition by Subject_Name,Date order by Marks desc) as rank from school)A where A.rank=1 order by Date")
    unified_obj2.write_parquet(highmark_df)
    unified_obj2.open_dir()

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'part-00001-612ed540-3560-4379-9656-765c267f9c4b-c000.snappy.parquet' -> 'highestmark.parquet'